In [1]:
import pandas as pd
import numpy as np
import pymmwr as pm
import datetime
import warnings
import io
import requests
warnings.simplefilter(action='ignore')

In [2]:
#Some common functions
def get_jhu_raw():
    url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
#     types = {
#         'FIPS': np.uint32
        
#     }
    return pd.read_csv(url)
df = get_jhu_raw()

In [3]:
url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
df = pd.read_csv(url)

# drop unnecessary columns
cols = ['UID', 'iso2', 'iso3', 'code3', 'Admin2']
df = df.drop(df[cols], axis=1)

state_agg = df.groupby(['Province_State']).sum()
us_nat = df.groupby(['Country_Region']).sum()
# display(df)
df_truth = df.drop(columns=df.columns[list(range(1,7))])
df_state_nat = state_agg.append(us_nat)
# display(df_truth)
df_truth_cumulative = df_truth
df_truth_incident = df_truth[df_truth.columns[1:]] - df_truth[df_truth.columns[1:]].shift(periods=1, axis='columns')

In [4]:
states = ['US', 'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut',
              'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
              'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri',
              'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
              'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island',
              'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
              'West Virginia', 'Wisconsin', 'Wyoming', 'District of Columbia']
df_orig = pd.read_csv(url)
df_orig = df_orig[df_orig['Province_State'].isin(states)]
# print(df_orig.columns[:30])
df_orig = df_orig[pd.notna(df_orig['FIPS'])]

## JHU truth data format

In [8]:
%pwd
fips_codes = pd.read_csv('../../data-locations/locations.csv')

In [9]:
def get_epi_data(date):
    format_str = '%m/%d/%y'  # The format
    dt = datetime.datetime.strptime(date, format_str).date()
    epi = pm.date_to_epiweek(dt)
    return epi.year, epi.week, epi.day

def configure_JHU_data(df, target):
    # convert matrix to repeating row format
    df_truth = df.unstack()
    df_truth = df_truth.reset_index()

    # get epi data from date
    df_truth['year'], df_truth['week'], df_truth['day'] = \
        zip(*df_truth['level_0'].map(get_epi_data))

    # rename columns
    df_truth = df_truth.rename(columns={0: "value",
                                        "level_1": "location_long"})

    # Get state IDs
    df_truth = df_truth.merge(fips_codes, left_on='location_long', right_on='location_name', how='left')

    # Drop NAs
    df_truth = df_truth.dropna(subset=['location', 'value'])

    # add leading zeros to state code
    df_truth['location'] = df_truth['location'].apply(lambda x: '{0:0>2}'.format(x))

    '''
    ####################################
    # Daily truth data output for reference
    ####################################
    '''

    # only output "location", "epiweek", "value"
    df_truth = df_truth.drop(['location_name'], axis=1)
    df_byday = df_truth.rename(columns={"level_0": "date", "location_long": "location_name"})

    # select columns
    df_byday = df_byday[["date", "location", "location_name", "value"]]

    # ensure value column is integer
    df_byday['value'] = df_byday['value'].astype(int)

    # change to yyyy/mm/dd format
    df_byday['date'] = pd.to_datetime(df_byday['date'])

    file_path = '../../data-truth/truth-' + target + '.csv'
    df_byday.to_csv(file_path, index=False)

    '''
    ####################################
    # Truth data output for visualization
    ####################################
    '''
    # Only visualize certain states
    states = ['US', 'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut',
              'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
              'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri',
              'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
              'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island',
              'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
              'West Virginia', 'Wisconsin', 'Wyoming', 'District of Columbia']
    df_truth = df_truth[df_truth["location_long"].isin(states)]

    # Observed data on the seventh day
    # or group by week for incident deaths
    if target == 'Incident Deaths':
        df_vis = df_truth.groupby(['week', 'location_long'], as_index=False).agg({'level_0': 'last',
                                                                                  'value': 'sum',
                                                                                  'year': 'last',
                                                                                  'day': 'last',
                                                                                  'location': 'last',
                                                                                  'abbreviation': 'last'})
        df_vis = df_vis[df_vis['day'] == 7]
    else:
        df_vis = df_truth[df_truth['day'] == 7]

    df_vis['week'] = df_vis['week'] + 1  # shift epiweek on axis

    # add leading zeros to epi week
    df_vis['week'] = df_vis['week'].apply(lambda x: '{0:0>2}'.format(x))

    # define epiweek
    df_vis['epiweek'] = df_vis['year'].astype(str) + df_vis['week']

    # Replace US with "nat" this is NECESSARY for visualization code!
    df_vis.loc[df_vis["location_long"] == "US", "abbreviation"] = "nat"

    # only output "location", "epiweek", "value"
    df_truth_short = df_vis[["abbreviation", "epiweek", "value"]]
    df_truth_short = df_truth_short.rename(columns={"abbreviation": "location"})

    df_truth_short["value"].replace({0: 0.1}, inplace=True)

    file_path = '../../visualization/vis-master/covid-csv-tools/dist/truth/' + target + '.json'
    # write to json
    with open(file_path, 'w') as f:
        f.write(df_truth_short.to_json(orient='records'))


In [10]:
# display(df_truth_cumulative)
configure_JHU_data(df_truth_cumulative.drop(columns=['FIPS']), "Cumulative Deaths")

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [13]:
def old_way():
    url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
    url_req = requests.get(url).content
    df = pd.read_csv(io.StringIO(url_req.decode('utf-8')))

    fips_codes = pd.read_csv('../../data-locations/locations.csv')

    
    # aggregate by state and nationally
    state_agg = df.groupby(['Province_State']).sum()
    us_nat = df.groupby(['Country_Region']).sum()
    df_state_nat = state_agg.append(us_nat)

    # drop unnecessary columns
    cols = list(range(0, 6))
    df_truth = df_state_nat.drop(df_state_nat.columns[cols], axis=1)

    df_truth_cumulative = df_truth
    df_truth_incident = df_truth - df_truth.shift(periods=1, axis='columns')

    display(df_truth_cumulative)
    configure_JHU_data(df_truth_cumulative, "Cumulative Deaths")
    configure_JHU_data(df_truth_incident, "Incident Deaths")
    return df_truth_cumulative


In [15]:
df_truth_states = old_way()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20
Alabama,0,0,0,0,0,0,0,0,0,0,...,2011,2013,2024,2037,2045,2076,2107,2152,2162,2182
Alaska,0,0,0,0,0,0,0,0,0,0,...,31,32,32,36,37,37,37,37,37,37
American Samoa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Arizona,0,0,0,0,0,0,0,0,0,0,...,4756,4771,4771,4792,4896,4929,4978,5007,5030,5029
Arkansas,0,0,0,0,0,0,0,0,0,0,...,674,687,696,711,732,739,756,772,784,797
California,0,0,0,0,0,0,0,0,0,0,...,12131,12153,12235,12382,12532,12677,12805,12894,12937,13022
Colorado,0,0,0,0,0,0,0,0,0,0,...,1918,1918,1919,1926,1927,1931,1937,1942,1942,1945
Connecticut,0,0,0,0,0,0,0,0,0,0,...,4460,4460,4460,4463,4463,4465,4465,4465,4465,4465
Delaware,0,0,0,0,0,0,0,0,0,0,...,600,600,604,603,604,604,604,604,604,605
Diamond Princess,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


KeyError: 'New Jersey'

# New way with county level forecasts

In [11]:
def preprocess_raw_jhu(df):
    cols_to_drop = ['UID', 'iso2', 'iso3', 'code3', 'Admin2', 'Country_Region', 
                    'Lat', 'Long_', 'Combined_Key', 'Population']
    df_final = df.drop(df[df['FIPS'].isna()].index)

    df_final.drop(columns=cols_to_drop, inplace=True)
    df_final['FIPS'] = df_final['FIPS'].astype(np.int32)
    fips_codes = pd.read_csv('../../data-locations/locations.csv')
    
    return df_final

df = preprocess_raw_jhu(get_jhu_raw())
df_truth = df.drop(columns=['Province_State']).set_index(['FIPS'])

state_fips = fips_codes[fips_codes['abbreviation'].notna()]
df_s_merged = df_truth_states.merge(state_fips, left_index=True, right_on='location_name', how='left')
df_states = df_s_merged.loc[df_s_merged.index.dropna()]
df_states.index = df_states.index.astype(np.int64)
df_states.sort_index(inplace=True)

NameError: name 'df_truth_states' is not defined

In [12]:
print(list(df_states.columns))
display(df_states)

NameError: name 'df_states' is not defined